In [46]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchsummary import summary
from atgen.layers import ActiSwitch

In [47]:
# Define a CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32*7*7, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x: torch.Tensor):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [48]:
# Hyperparameters
batch_size = 64
learning_rate = 0.001
num_epochs = 5

# Data loading and preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),          
    transforms.Normalize((0.1307,), (0.3081,))
])

# Download and load the training and test datasets
train_dataset = datasets.MNIST(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [49]:
# Model, loss function, and optimizer
model = SimpleCNN()
model = nn.Sequential(
    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    ActiSwitch(nn.ReLU(), True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(32),
    ActiSwitch(nn.ReLU(), True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0),
    nn.Flatten(),
    nn.Linear(32*7*7, 128),
    nn.BatchNorm1d(128),
    ActiSwitch(nn.ReLU(), True),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    ActiSwitch(nn.ReLU(), True),
    nn.Linear(64, 10),
)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Print model summary
model.to(device)
summary(model, input_size=(1, 28, 28))

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()               
        outputs = model(data)               
        loss = criterion(outputs, target)   
        loss.backward()                     
        optimizer.step()                    

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_loss:.4f}')



Layer (type:depth-idx)                   Param #
├─Conv2d: 1-1                            320
├─BatchNorm2d: 1-2                       64
├─ActiSwitch: 1-3                        --
|    └─ReLU: 2-1                         --
├─MaxPool2d: 1-4                         --
├─Conv2d: 1-5                            9,248
├─BatchNorm2d: 1-6                       64
├─ActiSwitch: 1-7                        --
|    └─ReLU: 2-2                         --
├─MaxPool2d: 1-8                         --
├─Flatten: 1-9                           --
├─Linear: 1-10                           200,832
├─BatchNorm1d: 1-11                      256
├─ActiSwitch: 1-12                       --
|    └─ReLU: 2-3                         --
├─Linear: 1-13                           8,256
├─BatchNorm1d: 1-14                      128
├─ActiSwitch: 1-15                       --
|    └─ReLU: 2-4                         --
├─Linear: 1-16                           650
Total params: 219,818
Trainable params: 219,818
Non-trai

Epoch 1/5: 100%|██████████| 938/938 [00:26<00:00, 34.95batch/s]


Epoch [1/5], Average Loss: 0.1245


Epoch 2/5: 100%|██████████| 938/938 [00:20<00:00, 44.97batch/s]


Epoch [2/5], Average Loss: 0.0488


...

Epoch [3/5], Average Loss: 0.0303


Epoch 4/5: 100%|██████████| 938/938 [00:20<00:00, 46.71batch/s]


Epoch [4/5], Average Loss: 0.0244


Epoch 5/5: 100%|██████████| 938/938 [00:19<00:00, 46.93batch/s]

Epoch [5/5], Average Loss: 0.0173


In [50]:
# Evaluate the model on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in tqdm(test_loader, desc="Evaluating", unit='batch'):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Accuracy of the model on the test set: {100 * correct / total:.2f}%')

Evaluating: 100%|██████████| 157/157 [00:01<00:00, 82.66batch/s]

Accuracy of the model on the test set: 98.82%
